In [1]:
from selenium import webdriver #selenium for chrome opening and log in
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd #pandas for dataframe
import numpy as np #numpy array
#import re
import lxml
import requests
from bs4 import BeautifulSoup #web scraping
import requests#get urls
from time import sleep
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [2]:
chrome_options = webdriver.ChromeOptions()
prefs = {"profile.default_content_setting_values.notifications" : 2}
chrome_options.add_experimental_option("prefs",prefs)

In [3]:
driver = webdriver.Chrome(ChromeDriverManager().install(),chrome_options=chrome_options)



====== WebDriver manager ======
Current google-chrome version is 97.0.4692
Get LATEST chromedriver version for 97.0.4692 google-chrome
Driver [C:\Users\omidg\.wdm\drivers\chromedriver\win32\97.0.4692.71\chromedriver.exe] found in cache
C:\Users\omidg\AppData\Local\Temp/ipykernel_19120/3257191672.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(),chrome_options=chrome_options)
C:\Users\omidg\AppData\Local\Temp/ipykernel_19120/3257191672.py:1: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(ChromeDriverManager().install(),chrome_options=chrome_options)


In [4]:
def hotels_table():
    query_result = driver.page_source
    soup = BeautifulSoup(query_result, 'html.parser')
    hots = soup.find_all(class_ = 'ui_column is-8 main_col allowEllipsis')
    df = pd.DataFrame(columns=['name','trip_link','website'])
    for i in hots:
        dic = {}
        name = i.find(class_='listing_title').get_text()[6:]
        print(name)
        link = 'https://www.tripadvisor.com' + i.find('a',href = True).get('href')
        try:
            website = 'https://www.tripadvisor.com' + i.find('div',class_='label isLink').get('data-link')
            driver.get(website)
            time.sleep(3)
            website = driver.current_url.split('/')[2]
        except:
            website = 'nothing'
        dic['name'] = name
        dic['trip_link'] = link
        dic['website'] = website
        df = df.append(dic, ignore_index=True)
    return (df)

In [5]:
def tripadvisor(city):
    driver.get('https://www.tripadvisor.com/Hotels')
    time.sleep(2)
    try:
        driver.find_element(By.XPATH,'//*[@id="onetrust-accept-btn-handler"]').click()
    except:
        pass
    driver.find_element(By.XPATH, "//input[@placeholder='Hotel name or destination']").send_keys(city)
    time.sleep(2)
    driver.find_element(By.XPATH,'//*[@id="typeahead_results"]/a[1]').click()
    time.sleep(2)
    driver.find_element(By.XPATH,'//*[@id="component_10"]/div/div[2]/div[10]/div[2]/div[1]/div').click()
    return (driver.current_url)

In [6]:
hotels = pd.DataFrame(columns=['name','trip_link','website'])
first_page = tripadvisor('Rome')
time.sleep(5)
first_link = driver.current_url

In [7]:
for i in range(3):
    df = hotels_table()
    hotels = pd.concat([hotels,df])
    driver.get(first_link)
    driver.find_element(By.XPATH,
                    '//*[@id="taplc_main_pagination_bar_hotels_less_links_v2_0"]/div/div/div/a').click()
    first_link = driver.current_url

hotels = hotels.reset_index(drop=True)

Singer Palace Hotel
NH Collection Roma Fori Imperiali
Rome Cavalieri, A Waldorf Astoria Hotel
NH Collection Roma Palazzo Cinquecento
Hotel de la Ville, A Rocco Forte Hotel
Anantara Palazzo Naiadi Rome Hotel
Hotel Vilon
Aleph Rome Hotel, Curio Collection by Hilton
Gasometer Urban Suites
Sofitel Roma Villa Borghese Hotel
Villa Agrippina Gran Melia
J.K. Place Roma
Hotel Indigo Rome - St. George, an IHG hotel
Grand Hotel Palace
Hotel Eden
Radisson Blu GHR Hotel Rome
Hotel de Russie
The First Roma Arte
The Inn At The Roman Forum
The Pantheon Iconic Rome Hotel
Navona Palace Luxury Inn
Hotel Splendide Royal
Hotel Hassler
Palazzo Montemartini Rome, A Radisson Collection Hotel
The H'All Tailor Suite
Eitch Borromini
Roma Luxus Hotel
Palazzo Dama Hotel
Rhona's Rooms B&B
Palm Suite
Singer Palace Hotel
NH Collection Roma Fori Imperiali
Rome Cavalieri, A Waldorf Astoria Hotel
NH Collection Roma Palazzo Cinquecento
Hotel de la Ville, A Rocco Forte Hotel
Anantara Palazzo Naiadi Rome Hotel
Hotel Vilon


In [55]:
hotels['phone'] = np.nan
hotels['email'] = np.nan
hotels['city'] = 'Rome'
hotels['address'] = np.nan
hotels['stars'] = '5'
for i in (hotels.index):
    driver.get(hotels.loc[i,'trip_link'])
    time.sleep(2)
    query_result = driver.page_source
    soup = BeautifulSoup(query_result, 'html.parser')
    data = soup.find_all(class_='bIWzQ')
    phone, email, address = 'nothing' , 'nothing', 'nothing'
    try:
        phone = data[0].get_text()
    except:
        pass
    try:
        email = data[1].get('href').split(':')[1]
    except:
        pass
    try:
        address = soup.find_all(class_='ceIOZ yYjkv')[0].get_text()
    except:
        pass
    try:
        if soup.find(class_='cISIs b').get_text() == 'Missing:':
            hotels.loc[i,'stars'] = '4'
    except:
        pass
            
    hotels.loc[i,'phone'] = phone
    hotels.loc[i,'email'] = email
    hotels.loc[i,'address'] = address

for i in (hotels.index):
    if hotels.loc[i,'website'] == 'nothing':
        if hotels.loc[i,'email'] != 'nothing':
            hotels.loc[i,'website'] = 'www.'+hotels.loc[i,'email'].split('@')[1]

In [57]:
hotels

,name,trip_link,website,phone,email,city,address,stars
0,Singer Palace Hotel,https://www.tripadvisor.com/Hotel_Review-g1877...,singerpalacehotel.com,+39 06 697 6161,singer@singerpalacehotel.com,Rome,"Via Alessandro Specchi 10, 00186 Rome Italy",5
1,NH Collection Roma Fori Imperiali,https://www.tripadvisor.com/Hotel_Review-g1877...,www.nh-hotels.com,0034916008321,customercare@nh-hotels.com,Rome,"Via di Santa Eufemia 19, 00187 Rome Italy",5
2,"Rome Cavalieri, A Waldorf Astoria Hotel",https://www.tripadvisor.com/Hotel_Review-g1877...,www.hilton.com,+39-06-35091,nothing,Rome,"Via Alberto Cadlolo 101, 00136 Rome Italy",5
3,NH Collection Roma Palazzo Cinquecento,https://www.tripadvisor.com/Hotel_Review-g1877...,www.nh-hotels.com,0034916008321,customercare@nh-hotels.com,Rome,"Piazza dei Cinquecento, 90, 00185 Rome Italy",5
4,"Hotel de la Ville, A Rocco Forte Hotel",https://www.tripadvisor.com/Hotel_Review-g1877...,www.roccofortehotels.com,+39 06 977931,reservations.delaville@roccofortehotels.com,Rome,"Via Sistina 69, 00187 Rome Italy",5
...,...,...,...,...,...,...,...,...
85,St. Anthony Guest House,https://www.tripadvisor.com/Hotel_Review-g1877...,nothing,nothing,nothing,Rome,"Piazza Asti 5, 00182 Rome Italy",5
86,Central Domus Roma,https://www.tripadvisor.com/Hotel_Review-g1877...,nothing,nothing,nothing,Rome,"Via Principe Amedeo 128, 00185 Rome Italy",5
87,Fendi Private Suites,https://www.tripadvisor.com/Hotel_Review-g1877...,www.fendiprivatesuites.com,nothing,nothing,Rome,"Via Della Fontanella Borghese 48, 00187 Rome I...",5
88,Residenza Venti Settembre,https://www.tripadvisor.com/Hotel_Review-g1877...,www.residenzaventisettembre.com,+390692114362,info@residenzaventisettembre.com,Rome,"Via XX Settembre, 58a, 00187 Rome Italy",4


In [56]:
hotels.to_csv('Roma_hotels_5.csv')